In [3]:
import meep as mp
import matplotlib.pyplot as plt
import numpy as np

# -----------------------------
# Simulation parameters
# -----------------------------
resolution = 5  # pixels/um

# Materials
gaas = mp.Medium(epsilon=12)

# Disk and waveguide geometry
disk_radius = 3.5     # um
disk_height = 0.5     # um
wg_length = 40        # um
wg_width = 0.22       # um
wg_height = 0.5       # um
gap = 0.08            # distance between disk and waveguides

# Simulation cell size (make sure divisible by resolution)
cell_x = int((wg_length + 10) * resolution) / resolution
cell_y = int((2*(disk_radius + gap + wg_width/2) + 10) * resolution) / resolution
cell_z = 4.0           # enough space above and below disk

cell = mp.Vector3(cell_x, cell_y, cell_z)

# PML layers
pml_layers = [mp.PML(2.0)]

# -----------------------------
# Geometry
# -----------------------------
geometry = [
    # Disk resonator
    mp.Cylinder(radius=disk_radius, height=disk_height, center=mp.Vector3(), material=gaas),

    # Top bus waveguide
    mp.Block(size=mp.Vector3(wg_length, wg_width, wg_height),
             center=mp.Vector3(0, disk_radius + gap + wg_width/2, wg_height),
             material=gaas),

    # Bottom drop waveguide
    mp.Block(size=mp.Vector3(wg_length, wg_width, wg_height),
             center=mp.Vector3(0, -disk_radius - gap - wg_width/2, wg_height),
             material=gaas)
]

# -----------------------------
# Source
# -----------------------------
source_x = -wg_length/2 + 1
source_y = disk_radius + gap + wg_width/2
source_z = 0

fmin = 0.3   # 1/um
fmax = 0.34
fcen = (fmin+fmax)/2
df = fmax - fmin
nfreq = 300

sources = [mp.Source(mp.GaussianSource(frequency=fcen, fwidth=df),
                     component=mp.Ez,
                     center=mp.Vector3(source_x, source_y, source_z),
                     size=mp.Vector3(0, wg_width, wg_height))]

# -----------------------------
# Flux monitors
# -----------------------------
flux_region_bus = mp.FluxRegion(center=mp.Vector3(wg_length/2 - 1, disk_radius + gap + wg_width/2, 0),
                                size=mp.Vector3(0, wg_width, wg_height))

flux_region_drop = mp.FluxRegion(center=mp.Vector3(wg_length/2 - 1, -disk_radius - gap - wg_width/2, 0),
                                 size=mp.Vector3(0, wg_width, wg_height))

# -----------------------------
# Simulation
# -----------------------------
sim = mp.Simulation(cell_size=cell,
                    geometry=geometry,
                    sources=sources,
                    boundary_layers=pml_layers,
                    resolution=resolution,
                    dimensions=3)

flux_bus = sim.add_flux(fcen, df, nfreq, flux_region_bus)
flux_drop = sim.add_flux(fcen, df, nfreq, flux_region_drop)

# -----------------------------
# Store Ez snapshots
# -----------------------------
ez_data = []

def store_fields(sim):
    ez_data.append(sim.get_array(center=mp.Vector3(), size=cell, component=mp.Ez))

# Run simulation with snapshots
sim.run(mp.at_every(20, store_fields),
        until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, mp.Vector3(0,0,0), 1e-3))

# -----------------------------
# Extract flux spectra
# -----------------------------
frequencies = np.array(mp.get_flux_freqs(flux_bus))
flux_bus_data = np.array(mp.get_fluxes(flux_bus))
flux_drop_data = np.array(mp.get_fluxes(flux_drop))

# -----------------------------
# Plot transmission spectra
# -----------------------------
plt.figure(figsize=(8,6))
plt.plot(frequencies, flux_bus_data, label="Bus Port")
plt.plot(frequencies, flux_drop_data, label="Drop Port")
plt.xlabel("Frequency (1/µm)")
plt.ylabel("Flux")
plt.title("3D Disk Resonator Transmission")
plt.legend()
plt.show()

# -----------------------------
# Plot Ez snapshots (mid z-plane)
# -----------------------------
mid_z_idx = int(cell_z/2 * resolution)  # index for mid-plane

for i, field in enumerate(ez_data[::5]):  # every 5th snapshot
    plt.figure(figsize=(7,6))
    plt.imshow(np.rot90(field[:,:,mid_z_idx]), cmap="RdBu", origin="lower",
               extent=[-cell_x/2, cell_x/2, -cell_y/2, cell_y/2])
    plt.colorbar(label="Ez field")
    plt.title(f"Ez snapshot {i*20} time units (mid z-plane)")
    plt.xlabel("x (µm)")
    plt.ylabel("y (µm)")
    plt.show()

# -----------------------------
# Plot geometry (mid z-plane)
# -----------------------------
eps = sim.get_array(center=mp.Vector3(), size=cell, component=mp.Dielectric)
plt.figure(figsize=(6,6))
plt.imshow(np.rot90(eps[:,:,mid_z_idx]), origin='lower', cmap='Greys')
plt.title("3D Geometry (mid z-plane)")
plt.xlabel("x (µm)")
plt.ylabel("y (µm)")
plt.show()


-----------
Initializing structure...


RuntimeError: meep: invalid boundary absorbers for this grid_volume

In [2]:
resolution = 8

cell_x = int((wg_length + 10) * resolution) / resolution
cell_y = int((2*(disk_radius + gap + wg_width/2) + 10) * resolution) / resolution
cell_z = 4.0  # finite disk height
cell = mp.Vector3(cell_x, cell_y, cell_z)

pml_layers = [mp.PML(2.0)]

geometry = [
    mp.Cylinder(radius=disk_radius, height=1.0, center=mp.Vector3(), material=gaas),
    mp.Block(size=mp.Vector3(wg_length, wg_width, 1.0),
             center=mp.Vector3(0, disk_radius + gap + wg_width/2, 0),
             material=gaas),
    mp.Block(size=mp.Vector3(wg_length, wg_width, 1.0),
             center=mp.Vector3(0, -disk_radius - gap - wg_width/2, 0),
             material=gaas)
]

sim = mp.Simulation(cell_size=cell,
                    geometry=geometry,
                    boundary_layers=pml_layers,
                    resolution=resolution,
                    dimensions=3)  # important for 3D!
